In [1]:
import tflearn
import glob
import string
import numpy as np
import io

def get_data(files) :
    global i, w, dictionary
    data = []
    for file in files :
        data_file = []
        with io.open(file, encoding='utf-8') as f:
            text = f.read().lower().encode('utf-8')
            words = text.split()
            for word in words:
                word = word.translate(None, string.punctuation)
                w = w + 1
                if w % 1000000 == 0 :
                    print w
                if word in dictionary:
                    index = dictionary[word]
                else:
                    index = i
                    dictionary[word] = i
                    i = i + 1
                data_file.append(index)
        data.append(data_file)
    return data

#Global variables
i = 0
w = 0
dictionary = dict()

#standard variables 
input_dim = 10000

# for training data
# for neg
path1 = './aclImdb/train/neg/*.txt'
files = glob.glob(path1)
print len(files)
train_X = get_data(files)
train_Y = np.full((2,len(train_X)),0)
train_Y[0,:] = 1

12500
1000000
2000000


In [2]:
# for pos
path1 = './aclImdb/train/pos/*.txt'
files = glob.glob(path1)
print len(files)
train  = get_data(files)
y = np.full((2, len(train)), 0)
y[1,:] = 1
print y.shape, train_Y.shape
#print [i for i in y if i == 1]
train_Y = np.append(train_Y, y, axis = 1)
print y.shape, train_Y.shape

#print [i for i in train_Y if i == 1]
train_X.extend(train)
print train_Y.shape
#for testing data
#for neg
path1 = './aclImdb/test/neg/*.txt'
files = glob.glob(path1)
print len(files)
test_X = get_data(files)
test_Y = np.full((2,len(test_X)),0)
test_Y[0,:] = 1
print test_Y.shape
#for pos
path1 = './aclImdb/test/pos/*.txt'
files = glob.glob(path1)
print len(files)
test  = get_data(files)
y = np.full((2,len(test)), 1)
y[1,:] = 1
test_Y = np.append(test_Y, y, axis = 1)
test_X.extend(test)

print len(dictionary), i, w, len(train_X), len(test_X)

12500
3000000
4000000
5000000
(2, 12500) (2, 12500)
(2, 12500) (2, 25000)
(2, 25000)
12500
6000000
7000000
8000000
(2, 12500)
12500
9000000
10000000
11000000
182730 182730 11557403 25000 25000


In [3]:
print train_Y.shape, test_Y.shape

(2, 25000) (2, 25000)


In [4]:
###### Get most frequent words

popular_words = sorted(dictionary, key = dictionary.get, reverse = False)

top = [dictionary[x] for x in popular_words[:input_dim]]

for i in range(len(train_X)) :
    train_X[i] = [x if x in top else 0 for x in train_X[i]]

for i in range(len(test_X)) :
    test_X[i] = [x if x in top else 0 for x in test_X[i]]

In [5]:
print len(top)
print len(train_X), len(test_X), type(train_Y), train_Y.shape, test_Y.shape
train_Y = train_Y.reshape(len(train_Y[1]), 2)
test_Y = test_Y.reshape(len(test_Y[1]), 2)
print train_Y.shape, test_Y.shape


10000
25000 25000 <type 'numpy.ndarray'> (2, 25000) (2, 25000)
(25000, 2) (25000, 2)


In [6]:
from tflearn.data_utils import to_categorical, pad_sequences

# Data preprocessing
# Sequence padding
trainX = pad_sequences(train_X, maxlen=100, value=0.)
testX = pad_sequences(test_X, maxlen=100, value=0.)

In [7]:
# Converting labels to binary vectors
trainY = to_categorical(train_Y[:, 0], nb_classes=2)
testY = to_categorical(test_Y[:, 0], nb_classes=2)

In [8]:
print(trainX.shape, trainY.shape, len(trainX), len(testY), train_Y.shape)

((25000, 100), (25000, 2), 25000, 25000, (25000, 2))


In [9]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

trainX, TrainY = unison_shuffled_copies(trainX, trainY)
testX, testY = unison_shuffled_copies(testX, testY)

In [10]:
########## Ready to build the network with train and test data #########

net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=input_dim, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX[:22500], trainY[:22500], validation_set=(trainX[22500:], trainY[22500:]), show_metric=True, batch_size=16)

predictions = model.predict(trainX)
print(predictions)

Training Step: 14069  | total loss: 0.16795 | time: 178.462s
| Adam | epoch: 010 | loss: 0.16795 - acc: 0.9440 -- iter: 22496/22500
Training Step: 14070  | total loss: 0.16283 | time: 183.914s
| Adam | epoch: 010 | loss: 0.16283 - acc: 0.9433 | val_loss: 2.12632 - val_acc: 0.4112 -- iter: 22500/22500
--
[[  3.49677750e-03   9.96503234e-01]
 [  1.32303648e-02   9.86769676e-01]
 [  6.19352423e-03   9.93806481e-01]
 ..., 
 [  8.70228469e-01   1.29771590e-01]
 [  9.60418165e-01   3.95818911e-02]
 [  3.34767319e-05   9.99966502e-01]]


In [26]:
testing_1 = glob.glob('./test*.txt')
print testing_1
testing_1 = get_data(testing_1)
print testing_1
for i in range(len(testing_1)) :
    testing_1[i] = [x if x in top else 0 for x in testing_1[i]]
print testing_1
testing_1 = pad_sequences(testing_1, maxlen=100, value=0.)
print(model.predict(testing_1))

['./test1.txt', './test.txt']
[[73, 546, 64798, 64799, 64800, 64801, 956, 30365, 64802, 3178, 993, 14695, 6, 10, 23774, 13615, 99, 191, 2036, 77, 12553, 39, 54, 2, 10157, 2190, 2, 217, 10597, 51, 20792, 30365, 14651, 2, 10, 1664, 1486, 1062, 323, 64, 6382, 217, 133, 217, 1619, 73, 64803, 51, 10, 878, 1438, 73, 2400, 34424, 32860, 962, 950, 39575, 6, 217, 3245, 14784, 10, 1664, 5, 24582, 39, 1848, 13160, 2753, 51, 10, 833, 1336, 1224, 73, 10, 380, 29344, 9, 878, 26736, 281, 1000, 37404, 5460, 13994, 29, 34424, 4878, 44, 11072, 6, 217, 4840, 51, 217, 12103, 323, 217, 7338, 617, 5, 10168, 6, 26, 472, 17372, 66, 1720, 577, 10, 1664, 51, 523, 25, 452, 2, 30365, 2, 3224, 10, 16615, 1530, 51, 2, 3931, 217, 64804, 73, 10, 1902, 5166, 173, 6208, 59266, 57900, 64805, 51, 6208, 64806, 64806, 57900, 64799, 2608, 168, 64807, 758, 5, 22, 8, 996, 26, 1684, 3256, 36, 187, 551, 12753, 1703, 29, 1324, 10, 202, 9, 4, 2608, 168, 825, 555, 1616, 87, 758, 16213, 314, 89, 16120, 92, 309, 30844, 64808, 9980, 